In [ ]:
import string
from nltk.tokenize import word_tokenize
from nltk.corpus import stopwords
from nltk.stem.porter import PorterStemmer
import lxml.html
import numpy as np
import nltk
nltk.download('punkt')
nltk.download('stopwords')

[nltk_data] Downloading package punkt to /root/nltk_data...
[nltk_data]   Unzipping tokenizers/punkt.zip.
[nltk_data] Downloading package stopwords to /root/nltk_data...
[nltk_data]   Unzipping corpora/stopwords.zip.


True

In [ ]:
import pandas as pd
import re
import matplotlib.pyplot as plt
from google.colab import drive
import csv
from pprint import pprint

In [ ]:
import gensim, spacy, logging, warnings
import gensim.corpora as corpora
from gensim.utils import lemmatize, simple_preprocess
from gensim.models import CoherenceModel
import matplotlib.pyplot as plt

In [ ]:
pip install emoji

In [ ]:
from emoji import UNICODE_EMOJI

In [ ]:
drive.mount('/content/gdrive')

Drive already mounted at /content/gdrive; to attempt to forcibly remount, call drive.mount("/content/gdrive", force_remount=True).


In [ ]:
df_ed = pd.read_csv(r'/content/gdrive/My Drive/Studies/Projects/PFE/PFE/Week_9/df_ed.csv')
df_rec = pd.read_csv(r'/content/gdrive/My Drive/Studies/Projects/PFE/PFE/Week_9/df_rec.csv')

In [ ]:
stop_words = stopwords.words('english')
stop_words.extend(['from', 'subject', 're', 'edu', 'use', 'not', 'would', 'say', 'could', '_', 'be', 'know', 'good', 'go', 'get', 'do', 'done', 'try', 'many', 'some', 'nice', 'thank', 'think', 'see', 'rather', 'easy', 'easily', 'lot', 'lack', 'make', 'want', 'seem', 'run', 'need', 'even', 'right', 'line', 'even', 'also', 'may', 'take', 'come'])

In [ ]:
def valid_text(text):
  if(text is None or text == '' or text == np.nan):
    return False
  else:
    return True

In [ ]:
def remove_tags(text):
  if (valid_text(text)):
    if (lxml.html.fromstring(text).find('.//*') is not None):
      return re.sub('<[^>]*>', '', text)
    else:
      return text

In [ ]:
def is_emoji(s):
  for emoji in UNICODE_EMOJI:
    if(emoji in str(s)):
      return True
    else:
      return False

In [ ]:
def deEmojify(s):
  if(is_emoji(s)):
    return s.encode('ascii', 'ignore').decode('ascii')
  else:
    return s

In [ ]:
#Split words
def splitting(text):
  tokens = ""
  try:
    tokens = word_tokenize(text)
  except:
    if(text == np.nan):
      print("nan")
    print("not nan")
  return tokens

In [ ]:
def remove_Nones(words):
  text = []
  for w in words:
    if (w != 'None'):
      text.append(w)
  return text

In [ ]:
def delinks(words):
  #text = [re.sub(r'''(?i)\b((?:(https|http)?://|www\d{0,3}[.]|[a-z0-9.\-]+[.][a-z]{2,4}/)(?:[^\s()<>]+|\(([^\s()<>]+|(\([^\s()<>]+\)))*\))+(?:\(([^\s()<>]+|(\([^\s()<>]+\)))*\)|[^\s`!()\[\]{};:'".,<>?«»“”‘’]))''', '', w, flags=re.MULTILINE) for w in words]
  text = [re.sub(r"http\S+", "", w) for w in words]
  return text

In [ ]:
def remove_numbers(words):
  output = [re.sub(r'\d+', '', w) for w in words]
  return output

In [ ]:
def depunct(tokens):
  # remove all tokens that are not alphabetic
  words = [word for word in tokens if word.isalpha()]
  table = str.maketrans('', '', string.punctuation)
  stripped = [w.translate(table) for w in words]
  return stripped

In [ ]:
def tolowercase(words):
  text = [w.lower() for w in words]
  return text

In [ ]:
def load_texts(text):
  inp = deEmojify(text)
  return splitting(inp)

In [ ]:
def pre_processing(l):
  no_none = remove_Nones(l)
  no_link = delinks(no_none)
  no_numb = remove_numbers(no_link)
  return (tolowercase(depunct(no_numb)))

In [ ]:
def dataframe_preprocessing(df):
  contents = []
  users = []
  for ind, row in df.iterrows():
    users.append(row["users"])
    #l = pre_processing(load_texts(row["content"]))
    txt = load_texts(row["content"])
    contents.append(pre_processing(txt))
  return users, contents

In [ ]:
u_ed, c_ed = dataframe_preprocessing(df_ed)

In [ ]:
u_rec, c_rec = dataframe_preprocessing(df_rec)

## Bigram / trigram

In [ ]:
# Build the bigram and trigram models
bigram_ed = gensim.models.Phrases(c_ed, min_count=5, threshold=100) # higher threshold fewer phrases.
#trigram_ed = gensim.models.Phrases(bigram_ed[c_ed], threshold=100)
bigram_mod_ed = gensim.models.phrases.Phraser(bigram_ed)
#trigram_mod_ed = gensim.models.phrases.Phraser(trigram_ed)

/usr/local/lib/python3.6/dist-packages/gensim/models/phrases.py:598: UserWarning: For a faster implementation, use the gensim.models.phrases.Phraser class
  warnings.warn("For a faster implementation, use the gensim.models.phrases.Phraser class")


In [ ]:
def process_words_ed(texts, stop_words=stop_words, allowed_postags=['NOUN', 'ADJ', 'VERB', 'ADV']):
    """Remove Stopwords, Form Bigrams, Trigrams and Lemmatization"""
    texts = [[word for word in simple_preprocess(str(doc)) if word not in stop_words] for doc in texts]
    texts = [bigram_mod_ed[doc] for doc in texts]
    #texts = [trigram_mod_ed[bigram_mod_ed[doc]] for doc in texts]
    texts_out = []
    nlp = spacy.load('en', disable=['parser', 'ner'])
    for sent in texts:
        doc = nlp(" ".join(sent))
        texts_out.append([token.lemma_ for token in doc if token.pos_ in allowed_postags])
    # remove stopwords once more after lemmatization
    texts_out = [[word for word in simple_preprocess(str(doc)) if word not in stop_words] for doc in texts_out]
    return texts_out

In [ ]:
data_ready_ed = process_words_ed(c_ed)  # processed Text Data!

In [ ]:
data_ready_ed

[['greek',
  'cal',
  'spinach',
  'salad',
  'cal',
  'sauce',
  'cup',
  'brown',
  'cal',
  'lunch',
  'leftover',
  'broccoli',
  'cal',
  'various',
  'snack',
  'work',
  'cal',
  'total',
  'really',
  'hate',
  'work',
  'restaurant',
  'anyone',
  'advice',
  'resist',
  'urge',
  'snack',
  'constantly',
  'food',
  'work',
  'tonight',
  'shall',
  'breakfast',
  'black',
  'coffee',
  'lunch',
  'spinach',
  'lite',
  'dress',
  'cal',
  'dinner',
  'sirracha',
  'lime',
  'popcorn',
  'cal',
  'total',
  'day',
  'today',
  'eat',
  'anything',
  'day',
  'hike',
  'boyfriend',
  'hour',
  'spinach',
  'salad',
  'couple',
  'cracker',
  'bbq',
  'salmon',
  'snack',
  'dinner',
  'together',
  'put',
  'total',
  'calorie',
  'misfit',
  'tracker',
  'net',
  'calorie',
  'day',
  'binge',
  'dinner',
  'consist',
  'frito',
  'bean',
  'dip',
  'fake',
  'crab',
  'feel',
  'shit',
  'fast',
  'today',
  'punish',
  'yesterday',
  'binge',
  'water',
  'breakfast',
  'lu

In [ ]:
# Create Dictionary
id2word_ed = corpora.Dictionary(data_ready_ed)

In [ ]:
# Create Corpus: Term Document Frequency
corpus_ed = [id2word_ed.doc2bow(text) for text in data_ready_ed]

In [ ]:
lda_model_ed = gensim.models.ldamodel.LdaModel(corpus=corpus_ed,
                                           id2word=id2word_ed,
                                           num_topics=8,
                                           random_state=100,
                                           update_every=1,
                                           chunksize=10,
                                           passes=10,
                                           alpha='symmetric',
                                           iterations=100,
                                           per_word_topics=True)


In [ ]:
# Build LDA model


In [ ]:
pprint(lda_model_ed.print_topics())

In [ ]:
def format_topics_sentences(ldamodel, corpus, texts, users):
    # Init output
    topics_df = pd.DataFrame()

    # Get main topic in each document
    for i, row_list in enumerate(ldamodel[corpus]):
        row = row_list[0] if ldamodel.per_word_topics else row_list
        # print(row)
        row = sorted(row, key=lambda x: (x[1]), reverse=True)
        # Get the Dominant topic, Perc Contribution and Keywords for each document
        for j, (topic_num, prop_topic) in enumerate(row):
            if j == 0:  # => dominant topic
                wp = ldamodel.show_topic(topic_num)
                topic_keywords = ", ".join([word for word, prop in wp])
                topics_df = topics_df.append(pd.Series([int(topic_num), round(prop_topic,4), topic_keywords]), ignore_index=True)
            else:
                break
    topics_df.columns = ['Dominant_Topic', 'Perc_Contribution', 'Topic_Keywords']

    # Add original text to the end of the output
    contents = pd.Series(texts)
    redditors = pd.Series(users)
    topics_df = pd.concat([topics_df, contents, redditors], axis=1)
    return(topics_df)

In [ ]:
df_topic_keywords_ed = format_topics_sentences(lda_model_ed, corpus_ed, c_ed, u_ed)

# Format
df_dominant_topic_ed = df_topic_keywords_ed.reset_index()
df_dominant_topic_ed.columns = ['Document_No', 'Dominant_Topic', 'Topic_Perc_Contrib', 'Keywords', 'Text', 'Redditors']

In [ ]:
df_dominant_topic_ed.head(10)

### recovery

In [ ]:
# Build the bigram and trigram models
bigram_rec = gensim.models.Phrases(c_rec, min_count=5, threshold=100) # higher threshold fewer phrases.
trigram_rec = gensim.models.Phrases(bigram_rec[c_rec], threshold=100)
bigram_mod_rec = gensim.models.phrases.Phraser(bigram_rec)
trigram_mod_rec = gensim.models.phrases.Phraser(trigram_rec)

In [ ]:
def process_words_rec(texts, stop_words=stop_words, allowed_postags=['NOUN', 'ADJ', 'VERB', 'ADV']):
    """Remove Stopwords, Form Bigrams, Trigrams and Lemmatization"""
    texts = [[word for word in simple_preprocess(str(doc)) if word not in stop_words] for doc in texts]
    texts = [bigram_mod_rec[doc] for doc in texts]
    texts = [trigram_mod_rec[bigram_mod_rec[doc]] for doc in texts]
    texts_out = []
    nlp = spacy.load('en', disable=['parser', 'ner'])
    for sent in texts:
        doc = nlp(" ".join(sent))
        texts_out.append([token.lemma_ for token in doc if token.pos_ in allowed_postags])
    # remove stopwords once more after lemmatization
    texts_out = [[word for word in simple_preprocess(str(doc)) if word not in stop_words] for doc in texts_out]
    return texts_out

In [ ]:
data_ready_rec = process_words_ed(c_rec)

In [ ]:
# Create Dictionary
id2word_rec = corpora.Dictionary(data_ready_rec)

In [ ]:
corpus_rec = [id2word_rec.doc2bow(text) for text in data_ready_rec]

In [ ]:
lda_model_rec = gensim.models.ldamodel.LdaModel(corpus=corpus_rec,
                                           id2word=id2word_rec,
                                           num_topics=15,
                                           random_state=100,
                                           update_every=1,
                                           chunksize=10,
                                           passes=10,
                                           alpha='symmetric',
                                           iterations=100,
                                           per_word_topics=True)

In [ ]:
pprint(lda_model_rec.print_topics())

In [ ]:
df_topic_keywords_rec = format_topics_sentences(lda_model_rec, corpus_rec, c_rec, u_rec)

# Format
df_dominant_topic_rec = df_topic_keywords_rec.reset_index()
df_dominant_topic_rec.columns = ['Document_No', 'Dominant_Topic', 'Topic_Perc_Contrib', 'Keywords', 'Text', 'Redditors']

In [ ]:
df_dominant_topic_rec.head(10)

## Visualizations

In [ ]:
pip install pyLDAvis

In [ ]:
import pyLDAvis.gensim
pyLDAvis.enable_notebook()
vis = pyLDAvis.gensim.prepare(lda_model_ed, corpus_ed, dictionary=lda_model_ed.id2word)
vis

In [ ]:
pyLDAvis.enable_notebook()
vis_rec = pyLDAvis.gensim.prepare(lda_model_rec, corpus_rec, dictionary=lda_model_rec.id2word)
vis_rec

In [ ]:
# Compute Coherence Score using c_v
coherence_model_lda = CoherenceModel(model=lda_model_ed, texts=data_ready_ed, dictionary=id2word_ed, coherence='c_v')
coherence_lda = coherence_model_lda.get_coherence()
print('\nCoherence Score: ', coherence_lda)

In [ ]:
coherence_model_lda = CoherenceModel(model=lda_model_ed, texts=data_ready_ed, dictionary=id2word_ed, coherence="u_mass")
coherence_lda = coherence_model_lda.get_coherence()
print('\nCoherence Score: ', coherence_lda)